<a href="https://colab.research.google.com/github/Tstrebe2/ml-mastery-transformers/blob/main/chapter-7/exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
from pandas import read_csv
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [2]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
  model = Sequential()
  model.add(SimpleRNN(hidden_units, input_shape=input_shape,
    activation=activation[0]))
  model.add(Dense(units=dense_units, activation=activation[1]))
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

demo_model = create_RNN(2, 1, (3, 1), activation=['linear', 'linear'])

In [7]:
wx = demo_model.get_weights()[0]
wh = demo_model.get_weights()[1]
bh = demo_model.get_weights()[2]
wy = demo_model.get_weights()[3]
by = demo_model.get_weights()[4]

In [8]:
print('wx = ', wx, ' wh = ', wh, ' bh = ', bh, ' wy =', wy, 'by = ', by)

wx =  [[-1.2746286  -0.14352417]]  wh =  [[ 0.7235912  -0.69022876]
 [-0.69022876 -0.7235912 ]]  bh =  [0. 0.]  wy = [[0.4827776 ]
 [0.33579004]] by =  [0.]


In [22]:
x = np.array([1, 2, 3])
x_input = x.reshape((1, 3, 1))

y_pred_model = demo_model.predict(x_input)

m = 2
h0 = np.zeros(m)
h1 = np.dot(x[0], wx) + h0 + bh
h2 = np.dot(x[1], wx) + np.dot(h1, wh) + bh
h3 = np.dot(x[2], wx) + np.dot(h2, wh) + bh
o3 = np.dot(h3, wy) + by

print('h1 =', h1, 'h2 =', h2, 'h3 =', h3)
print("Prediction from network ", y_pred_model)
print("Prediction from our computation ", o3)

1/1 [==============================] - 0s 17ms/step
h1 = [[-1.27462864 -0.14352417]] h2 = [[-3.37250285  0.69658983]] h3 = [[-6.74500566  1.39317967]]
Prediction from network  [[-2.7885218]]
Prediction from our computation  [[-2.78852176]]
